In [1]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import numpy as np
import json
import requests
from pprint import pprint as pprint
import polyline
import folium

In [3]:
host="http://tiramisu.hopto.org:2005"

In [4]:
Zürich_HB = {'lat': '47.377', 'lon':'8.54'}
cividi = {'lat':'47.3835', 'lon':'8.4966'}

# Optimized Route service

In [5]:
def routing(start, end, host=host, costing='pedestrian'):
    headers = {'Content-type': 'application/json'}
    data = json.dumps({"locations": [start, end], "costing": costing})
    url = host + '/route'
    return requests.post(url=url, data=data, headers=headers)

In [6]:
start = Zürich_HB
end = cividi
routing_request = routing(start, end, costing='pedestrian')
if routing_request.ok:
    routing_result= routing_request.json()
    print('Successfully')
else: 
    pprint(routing_request.json())

Successfully


In [7]:
distance = routing_result['trip']['summary']['length']
dwelltime = routing_result['trip']['summary']['time']
print('that route as %d km long and took %d s' % (distance, dwelltime))

that route as 4 km long and took 2903 s


In [8]:
routing_points = polyline.decode(routing_result['trip']['legs'][0]['shape'], precision=6)

In [9]:
routing_lats = [result[0] for result in routing_points]
routing_lons = [result[1] for result in routing_points]

In [10]:
m = folium.Map()
folium.PolyLine(routing_points).add_to(m)
folium.Marker((start['lat'], start['lon']),icon=folium.Icon(icon="play")).add_to(m)
folium.Marker((end['lat'], end['lon']),icon=folium.Icon(icon="stop")).add_to(m)
m.fit_bounds([[min(routing_lats), min(routing_lons)],[max(routing_lats), max(routing_lons)]])
m

# Isochrone & Isodistance service

In [11]:
def isochrone(start, host=host, costing='pedestrian'):
    headers = {'Content-type': 'application/json'}
    data = json.dumps({
        "locations": [start],
        "costing": costing,
        "contours": [{
            "time": 30,
            "color": "fc0000"
        }],
        "polygons": True

    })
    url = host + '/isochrone'
    return requests.post(url=url, data=data, headers=headers)

In [12]:
start = cividi
isochrone_request = isochrone(start,costing='pedestrian')
if isochrone_request.ok:
    isochrone_result= isochrone_request.json()
    print('Successfully')
else: 
    pprint(isochrone_request.json())

Successfully


In [13]:
isochrone_lons = [result[0] for result in isochrone_result['features'][0]['geometry']['coordinates'][0]]
isochrone_lats = [result[1] for result in isochrone_result['features'][0]['geometry']['coordinates'][0]]

In [14]:
m = folium.Map()
folium.GeoJson(isochrone_result).add_to(m)
folium.Marker(location=(start['lat'], start['lon']),icon=folium.Icon(icon="play")).add_to(m)
m.fit_bounds([[min(isochrone_lats), min(isochrone_lons)],[max(isochrone_lats), max(isochrone_lons)]])
m